In [5]:
from collections import defaultdict
from matplotlib import pyplot as plt
import numpy as np
import operator
import pickle

import src.fusiontools as fusiontools
import src.scoring as scoring
import src.postprocess as postprocess

%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
path_to_results = 'results/'

# Load best models

In [24]:
# Load best recency model
recency_rank_dic_path = 'ranks_test/recency_beta_7000_rec-all'
with open(recency_rank_dic_path, 'rb') as infile:
    recency_ranks = pickle.load(infile)
    
# Load best knn model
knn_rank_dic_path = 'ranks_test/knn-k-50-rec-all'
with open(knn_rank_dic_path, 'rb') as infile:
    knn_ranks = pickle.load(infile)

# Load best bayesian model
bayes_rank_dic_path = 'ranks_test/bayesian_results_0-35785'
with open(bayes_rank_dic_path, 'rb') as infile:
    bayes_ranks = pickle.load(infile)
    
# # Load best graph_through_words model
words_rank_dic_path = 'ranks_test/resjo_pickle_318'
with open(words_rank_dic_path, 'rb') as infile:
    words_ranks = pickle.load(infile)

# Load best graph_through_words model
centroids_rank_dic_path = 'ranks_test/centroids_dict_test'
with open(centroids_rank_dic_path, 'rb') as infile:
    centroids_ranks = pickle.load(infile)

# Discard tail of recipients

In [25]:
nb_kept_recips = 100
knn_ranks_cropped = fusiontools.keep_only_max_recips(knn_ranks, nb_kept_recips)
recency_ranks_cropped = fusiontools.keep_only_max_recips(recency_ranks, nb_kept_recips)
bayes_ranks_cropped = fusiontools.keep_only_max_recips(bayes_ranks, nb_kept_recips)
centroids_ranks_cropped = fusiontools.keep_only_max_recips(centroids_ranks, nb_kept_recips)


words_ranks_cropped = fusiontools.keep_only_max_recips(words_ranks, nb_kept_recips)

# Prepare models and weights

In [34]:
# models = [bayes_ranks_cropped, recency_ranks_cropped, knn_ranks_cropped]
# weights = [0.35785, 0.39316, 0.20716]


# models = [bayes_ranks_cropped, recency_ranks_cropped, words_ranks_cropped]
# weights = [0.35785, 0.39316, 0.33785]

# models = [bayes_ranks_cropped, recency_ranks_cropped, words_ranks_cropped, knn_ranks_cropped]
# weights = [0.35785*0.6, 0.39316*0.5, 0.33785*0.6, 0.20716*0.2]

# models = [knn_ranks_cropped, recency_ranks_cropped, bayes_ranks_cropped, centroid_ranks_cropped, words_ranks_cropped]
#val 0.5, 1, 0.5, 1, 0.5,
models = [bayes_ranks_cropped, centroids_ranks_cropped, recency_ranks_cropped, knn_ranks_cropped, words_ranks_cropped]
#weights = [0.35785, 0.24153, 0.39316, 0.20716, 0.33785]
weights = [0.35785/2, 0.24153, 0.39316, 0.20716/2, 0.33785/2]

#Fast check that dictionnaries are the right lenght
for model in models:
    print(len(model))

2362
2362
2362
2362
2362


In [23]:
for mid, recipients in words_ranks_cropped.items():
    print(len(recipients))


100
1
100
14
14
100
100
53
100
100
100
58
100
100
100
100
100
100
100
100
80
100
100
100
100
100
77
100
100
81
100
100
100
100
100
100
100
100
100
100
100
100
98
100
100
100
95
100
14
100
100
100
100
88
100
100
100
16
100
100
100
100
100
100
100
100
87
100
100
100
100
100
100
100
14
100
100
77
100
100
82
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
44
72
100
100
62
67
100
100
45
46
100
51
89
100
100
100
100
100
100
100
100
100
100
0
100
100
82
100
100
67
100
100
100
100
100
100
100
77
100
100
97
100
100
100
100
1
100
36
100
57
100
100
100
100
100
100
100
100
100
100
100
100
41
100
100
100
100
100
100
100
1
100
100
100
100
100
100
94
100
100
100
100
100
100
100
100
83
100
100
34
100
21
100
100
100
100
100
98
100
100
100
100
100
100
100
100
100
100
100
100
100
100
77
76
98
41
100
100
100
100
100
86
100
100
100
36
36
36
100
100
36
44
94
100
100
100
96
100
100
100
100
100
100
96
100
100
100
100
100
100
100
100
51
100
95
0
100
14
77
62
100
100
100
100
100
100
51
8

In [35]:
# models = [knn_ranks, recency_ranks]
# weights = [0.20716, 0.39316]

# models = [bayes_100, recency_ranks]
# weights = [0.34218, 0.39316]



reranking_coeff = 12
fusion_dic = fusiontools.reciprocal_rerank(models, reranking_coeff,  weights=weights)
# print(fusion_dic)

            
                

In [36]:
postprocess.write_results_ranked(fusion_dic, path_to_results, 'fusion-re-weighted-reranking-12-bayes-recency-tfidf-knn-centroids-words.csv')